In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By

browser = webdriver.Chrome()  # 경로지정
browser.get("http://google.com")


browser.find_element(By.CLASS_NAME,'uU7dJb').text


'대한민국'

In [11]:
browser = webdriver.Chrome()  # 경로지정
url ="https://www.yes24.com/Product/Category/BestSeller?CategoryNumber=001&sumgb=06"
browser.get(url)

In [12]:
# 1페이지의 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터
# browser.find_element(By.CLASS_NAME, 'gd_name') # element: 요소
datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements: 리스트

for i in datas:
    print(i.get_attribute('href'))

https://www.yes24.com/Product/Goods/133213071
https://www.yes24.com/Product/Goods/125557465
https://www.yes24.com/Product/Goods/131930939
https://www.yes24.com/Product/Goods/133312039
https://www.yes24.com/Product/Goods/133870359
https://www.yes24.com/Product/Goods/104663596
https://www.yes24.com/Product/Goods/133236068
https://www.yes24.com/Product/Goods/132560712
https://www.yes24.com/Product/Goods/133199014
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/133308510
https://www.yes24.com/Product/Goods/134331993
https://www.yes24.com/Product/Goods/128266166
https://www.yes24.com/Product/Goods/124999476
https://www.yes24.com/Product/Goods/124539805
https://www.yes24.com/Product/Goods/8759796
https://www.yes24.com/Product/Goods/129602366
https://www.yes24.com/Product/Goods/132565140
https://www.yes24.com/Product/Goods/133819718
https://www.yes24.com/Product/Goods/118322901
https://www.yes24.com/Product/Goods/133138701
https://www.yes24.com/Product/Goods/

In [17]:
import time

link_list = []
for i in range (1,2):
    print('*' * 10, f'현재 {i} 페이지 수집 중 입니다.', '*' * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    time.sleep(1)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
    ### 1페이지 전체의 링크 데이터
    # # browser.find_element(By.CLASS_NAME, 'gd_name') # element: 요소
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements: 리스트
    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)


print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/133213071', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/131930939', 'https://www.yes24.com/Product/Goods/133312039', 'https://www.yes24.com/Product/Goods/133870359', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/133236068', 'https://www.yes24.com/Product/Goods/132560712', 'https://www.yes24.com/Product/Goods/133199014', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/133308510', 'https://www.yes24.com/Product/Goods/134331993', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/8759796', 'https://www.yes24.com/Product/Goods/129602366', 'https://www.yes24.com/Product/Goods/132565140', 'https://www.yes24.com/Product/Goods/133819718', 'https://www.yes24.com/Produc

In [18]:
len(link_list)

24

In [15]:
# for link in link_list:
    # 상세 페이지로 이동
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text

autor = browser.find_element(By.CLASS_NAME, 'gd_pubArea').text

publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

rating = browser.find_element(By.CLASS_NAME, 'yes_b').text

reviews = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text

sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split (" ")[2]

price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]

ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]

In [21]:
import pymysql
import re
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException


conn = pymysql.connect(
    host='localhost',
    user='root',
    password='4463',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for link in link_list:
        browser.get(link)
        time.sleep(0.1)
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
    
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"
            
        try:  
            rating = browser.find_element(By.CLASS_NAME, 'gd_rating').find_element(By.CLASS_NAME, 'yes_b').text
        except NoSuchElementException:
            rating = 0
        try:
            review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.CLASS_NAME, 'txC_blue').text
            review = int(review.replace(",",""))
        except NoSuchElementException:
            review = 0

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))
        
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))
        
        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")
            
        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part))
            except:
                ranking = 0
                
            print(ranking)
            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0
        
        sql = """INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )"""
        
        cur.execute(sql,( title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
